# LLMのRAGを使ったQABOT

1. **データ取得**:
    - ICML 2024の論文PDFを取得し、./icml_2024_papersディレクトリに格納する
2. **データの埋め込みベクトル化**:
    - 取得したPDFからテキストを抽出し、ベクトル化する。    <---- now
3. **検索インデックスの構築**:
    - ベクトル化されたデータを元に検索インデックスを構築する。
4. **LLMのRAGモデルの設定**:
    - 質問に対して関連する文書を検索し、その文書を基に回答を生成するRAGモデルを設定する。
5. **QABOTの構築**:
    - ユーザーインターフェースを作成し、ユーザーからの質問に対してRAGモデルを使って回答するQABOTを構築する。

In [1]:
!pip3 install python-dotenv tiktoken
!pip3 install openai==1.3.4
!pip3 install PyPDF2

In [2]:
import os
from PyPDF2 import PdfReader
import pandas as pd
import tiktoken
import openai
from tqdm import tqdm
import time

import openai
from openai import OpenAI
from dotenv import load_dotenv

# すべての警告を無視する
# import warnings
# warnings.filterwarnings('ignore')

In [3]:
load_dotenv()

True

In [4]:
openai.api_key = os.getenv("OPENAI_API_KEY")

MODEL_NAME = "gpt-3.5-turbo-0125"
# MODEL_NAME = "gpt-3.5-turbo-instruct"
# MODEL_NAME = "gpt-4-0125-preview"
# MODEL_NAME = "gpt-4-turbo-2024-04-09"
MODEL4o_NAME = "gpt-4o-2024-05-13"

# モデルとエンコーディングの設定
embedding_model = "text-embedding-3-small"
embedding_encoding = "cl100k_base"
max_tokens = 8000


TEMPERATURE = 0.7

# OpenAIクライアントの初期化
client = OpenAI()

In [5]:
# PDFディレクトリと保存ディレクトリ
pdf_dir = "./icml_2024_papers"
save_dir = "./icml_2024_embeddings"
if not os.path.exists(save_dir):
    os.makedirs(save_dir)

In [ ]:
def convert_pdf_to_text(pdf_path):
    reader = PdfReader(pdf_path)
    text = ''
    for page in reader.pages:
        text += page.extract_text() + '\n'
    return text

In [ ]:
def chunk_text(text, max_tokens, encoding):
    tokens = encoding.encode(text)
    chunks = []
    for i in range(0, len(tokens), max_tokens):
        chunk = tokens[i:i + max_tokens]
        chunks.append(encoding.decode(chunk))
    return chunks

In [ ]:
def get_embedding(text, model):
    text = text.replace("\n", " ")
    response = client.embeddings.create(input=[text], model=model)
    return response.data[0].embedding

In [ ]:
# エンコーディングの取得
encoding = tiktoken.get_encoding(embedding_encoding)

In [ ]:
# PDFファイルごとに処理
for pdf_file in tqdm(os.listdir(pdf_dir), desc="Total Progress"):
    if pdf_file.endswith(".pdf"):
        try:
            pdf_path = os.path.join(pdf_dir, pdf_file)
            text = convert_pdf_to_text(pdf_path)
            chunks = chunk_text(text, max_tokens, encoding)
            
            embeddings = []
            for chunk in chunks:
                embedding = get_embedding(chunk, model=embedding_model)
                embeddings.append(embedding)
                time.sleep(1)  # ダウンロード間隔を1秒に設定

            # ベクトルを保存
            base_name = os.path.splitext(pdf_file)[0]
            save_path = os.path.join(save_dir, f"{base_name}_embeddings.csv")
            df = pd.DataFrame(embeddings)
            df.to_csv(save_path, index=False)
            print(f"Processed and saved embeddings for {pdf_file}")
        except Exception as e:
            print(f"Failed to process {pdf_file}: {e}")

Total Progress:  10%|█         | 1/10 [00:09<01:28,  9.84s/it]

Processed and saved embeddings for A Bias-Variance-Covariance Decomposition of Kernel Scores for Generative Models.pdf


unknown widths : 
[0, IndirectObject(746, 0, 140368358051456), 9, 10, 284, 11, IndirectObject(747, 0, 140368358051456), 17, 26, 513, 27, 28, 207, 29, 31, 596, 32, IndirectObject(748, 0, 140368358051456), 97, 98, 513, 99, IndirectObject(749, 0, 140368358051456), 100, 101, 513, 102, IndirectObject(750, 0, 140368358051456), 107, 108, 255, 109, 110, 523, 111, 113, 500, 114, IndirectObject(751, 0, 140368358051456), 122, IndirectObject(752, 0, 140368358051456), 124, 136, 300, 138, IndirectObject(753, 0, 140368358051456), 171, 176, 612, 177, IndirectObject(754, 0, 140368358051456), 178, 181, 492, 182, 185, 239, 186, IndirectObject(755, 0, 140368358051456), 187, 191, 689, 192, IndirectObject(756, 0, 140368358051456), 193, 196, 647, 197, 198, 541, 199, IndirectObject(757, 0, 140368358051456), 200, 205, 482, 206, IndirectObject(758, 0, 140368358051456), 207, 210, 501, 211, 214, 234, 215, IndirectObject(759, 0, 140368358051456), 216, 220, 549, 221, IndirectObject(760, 0, 140368358051456), 222, 22

Processed and saved embeddings for A Closer Look at the Limitations of Instruction Tuning.pdf


Total Progress:  30%|███       | 3/10 [00:26<00:56,  8.09s/it]

...

Processed and saved embeddings for Active Preference Learning for Large Language Models.pdf


Total Progress:  90%|█████████ | 9/10 [00:57<00:05,  5.70s/it]

Processed and saved embeddings for Adaptive Text Watermark for Large Language Models.pdf


Total Progress: 100%|██████████| 10/10 [01:01<00:00,  6.13s/it]

Processed and saved embeddings for Zero-Shot ECG Classification with Multimodal Learning and Test-time Clinical Knowledge Enhancement.pdf


In [11]:
import pandas as pd

In [12]:
# ディレクトリ内のファイルをリスト化
files = os.listdir(save_dir)
files

['A Bias-Variance-Covariance Decomposition of Kernel Scores for Generative Models_embeddings.csv',
 'A Closer Look at the Limitations of Instruction Tuning_embeddings.csv',
 'A Human-Inspired Reading Agent with Gist Memory of Very Long Contexts_embeddings.csv',
 'A Language Model’s Guide Through Latent Space_embeddings.csv',
 'A Multimodal Automated Interpretability Agent_embeddings.csv',
 'A Sober Look at LLMs for Material Discovery: Are They Actually Good for Bayesian Optimization Over Molecules?_embeddings.csv',
 'A Tale of Tails: Model Collapse as a Change of Scaling Laws_embeddings.csv',
 'Active Preference Learning for Large Language Models_embeddings.csv',
 'Adaptive Text Watermark for Large Language Models_embeddings.csv',
 'Zero-Shot ECG Classification with Multimodal Learning and Test-time Clinical Knowledge Enhancement_embeddings.csv']

In [14]:
# ファイルが存在するか確認
if files:
    file_name = files[0]  # 最初のファイルを選択
    file_path = os.path.join(save_dir, file_name)

    # ファイルを読み込んで表示
    if os.path.exists(file_path):
        df = pd.read_csv(file_path)
        pd.set_option('display.max_columns', None)  # すべての列を表示
        print(f"Displaying contents of file: {file_name}")
    else:
        print(f"File {file_path} does not exist.")
else:
    print(f"No files found in directory {save_dir}.")

Displaying contents of file: A Bias-Variance-Covariance Decomposition of Kernel Scores for Generative Models_embeddings.csv


In [15]:
df.head()

0         1         2         3         4         5         6  \
0  0.013282  0.028448  0.041178  0.018329 -0.010717  0.005605  0.016978   
1  0.028001  0.019591  0.041232  0.019517  0.011071  0.023692  0.019286   
2  0.015863  0.001282  0.010860  0.031395  0.009899  0.007963  0.025310   
3  0.019278 -0.005741  0.012253  0.012427  0.006656  0.005463  0.021490   
4  0.016145  0.018765  0.030035  0.005577 -0.003394  0.004702  0.044087   

          7         8         9        10        11        12        13  \
0  0.006485 -0.009385  0.045959  0.027123 -0.006050 -0.037151 -0.004001   
1  0.001311 -0.037863  0.039914  0.036423 -0.011822 -0.037375  0.018639   
2 -0.025768 -0.031370  0.042930  0.025552 -0.009319 -0.026456  0.024622   
3 -0.030003 -0.031960  0.036304  0.019559 -0.016650 -0.016945  0.025807   
4 -0.030767 -0.017652  0.019145  0.029025 -0.014893 -0.016189  0.045404   

         14        15        16        17        18        19        20  \
0 -0.001454  0.007664  0.010970 -0.008424 -0.026032  0.003634 -0.016588   
1 -0.011083  0.007391  0.034470  0.000484 -0.017479  0.023204 -0.016698   
2  0.002183  0.003009  0.033025  0.021796 -0.036691  0.015468 -0.008002   
3  0.022589  0.017549 -0.000370 -0.002554 -0.041076  0.042417 -0.006609   
4  0.003934  0.008314  0.023244  0.005646 -0.048303  0.037793  0.000669   

         21        22        23        24        25        26        27  \
0 -0.024369  0.002238  0.002465  0.034631 -0.016263  0.002981  0.042270   
1 -0.030686  0.005975  0.023253  0.046456 -0.023985 -0.001646  0.006341   
2 -0.014259  0.011229  0.016640  0.013406 -0.030148  0.007531  0.000684   
3 -0.023514  0.033327 -0.002359  0.002438 -0.037135 -0.003583  0.002202   
4 -0.047249  0.042389  0.008709  0.019336 -0.017521 -0.003762 -0.013635   

         28        29        30        31        32        33        34  \
0  0.034995 -0.048609  0.027980 -0.024538 -0.030475 -0.054038 -0.009236   
1  0.043478 -0.071479  0.002601 -0.019517 -0.029710 -0.057124 -0.035422   
2  0.038474 -0.065846  0.041046 -0.027041  0.002406 -0.067374 -0.022165   
3  0.061882 -0.052659  0.048664 -0.019680  0.014948 -0.064510 -0.010142   
4  0.044555 -0.051757  0.031031 -0.032758  0.006009 -0.071371 -0.022936   

         35        36        37        38        39        40        41  \
0  0.083344  0.020680  0.010275 -0.012646 -0.000958 -0.006865 -0.058247   
1  0.075580 -0.012890  0.015441 -0.039816  0.022386 -0.011132 -0.031052   
2  0.085605 -0.007314  0.016742 -0.025284  0.023820  0.012598 -0.031268   
3  0.098829 -0.034534  0.036652 -0.023890  0.016302  0.027120  0.002500   
4  0.057641 -0.031850  0.035158 -0.039784  0.038759  0.020229  0.021619   

         42        43        44        45        46        47        48  \
0 -0.036970  0.058767 -0.018706  0.010535  0.002082 -0.024798  0.012107   
1 -0.070746  0.060981 -0.011498 -0.019237  0.002629 -0.023228  0.034348   
2 -0.094466  0.018830 -0.000894 -0.011108 -0.036590 -0.051587  0.032592   
3 -0.097274 -0.025766  0.002931 -0.017723 -0.033220 -0.019090  0.038395   
4 -0.080329 -0.022644 -0.014352 -0.005573 -0.047571 -0.017213  0.007227   

         49        50        51        52        53        54        55  \
0  0.028760 -0.060897  0.025967  0.006544 -0.052480 -0.001549 -0.016484   
1  0.000304 -0.048604  0.012365  0.028391 -0.025682 -0.022227 -0.005804   
2  0.018995 -0.039645 -0.008638  0.015799 -0.010242 -0.048481  0.001658   
3 -0.005021 -0.038422  0.012776  0.005252 -0.018058 -0.019962 -0.002297   
4  0.005690 -0.033343 -0.009404  0.001156 -0.014300 -0.026039  0.014293   

         56        57        58        59        60        61        62  \
0 -0.038554  0.001075  0.011840 -0.070302  0.042945 -0.018212  0.053363   
1 -0.018163  0.010979  0.011474 -0.064106  0.024583 -0.024107  0.033152   
2 -0.029002  0.039085  0.016945 -0.071040  0.039416 -0.009510  0.034222   
3 -0.010075  0.044803  0.037269 -0.064885  0.071696  0.013299  0.044160   
4 -0.028455  0.039696  0.03